## 1. Testing

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df = pd.read_csv('../../flight-delay/extracted_data/mysql/db/2003.csv')
df.head()

In [ ]:
df2=df[["Cancelled", "CancellationCode", "Diverted"]].drop_duplicates()
len(df2)

In [ ]:
df2.head()

In [ ]:
df3=df[["Cancelled","CancellationCode", "Diverted"]].drop_duplicates()
len(df3)

## 2. Load and integrate data

In [ ]:
# import libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# initialize spark session
spark = SparkSession \
        .builder \
        .master("local[*]") \
        .appName("StagingToWarehouse") \
        .config("spark.executor.memory", "2g") \
        .config("spark.executor.cores", '2') \
        .config("spark.driver.memory", '2g') \
        .config("spark.driver.cores", '2') \
        .config("spark.cassandra.connection.host", "localhost") \
        .config("spark.cassandra.connection.port", "9042") \
        .config("spark.cassandra.auth.username", "root") \
        .config("spark.cassandra.auth.password", "admin") \
        .config("spark.jars", "../../tools/spark-jars/spark-3.2-bigquery-0.31.1.jar") \
        .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
        .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "../../tools/bigquery/key_file.json") \
        .getOrCreate()

In [ ]:
# get max id of fact_records table
fact_records_max_id = spark.read \
                        .format("bigquery") \
                        .options(parentProject="test-373705") \
                        .options(table="flight_delay.fact_records") \
                        .load().select("id") \
                        .agg(max("id")) \
                        .collect()[0][0]
# ensure max id have a value
if fact_records_max_id is None:
    fact_records_max_id=-1

# extract data from mysql table
mysql = spark.read \
            .format("org.apache.spark.sql.cassandra") \
            .options(keyspace="flight_delay") \
            .options(table="mysql") \
            .load() \
            .filter(col("id")>fact_records_max_id)
# extract data from mongodb table
mongodb = spark.read \
            .format("org.apache.spark.sql.cassandra") \
            .options(keyspace="flight_delay") \
            .options(table="mongodb") \
            .load() \
            .filter(col("id")>fact_records_max_id)
# integrate data, rename columns, and change datatype
source = mysql.union(mongodb).select(
    col("id").alias("id"),
    col("actualelapsedtime").cast("bigint").alias("actual_elapsed_time"),
    col("airtime").cast("bigint").alias("air_time"),
    col("arrdel15").cast("bigint").alias("arr_delay_15"),
    col("arrdelay").cast("bigint").alias("arr_delay"),
    col("arrdelayminutes").cast("bigint").alias("arr_delay_minutes"),
    col("arrivaldelaygroups").cast("bigint").alias("arr_delay_groups"),
    col("arrtime").cast("bigint").alias("arr_time"),
    col("arrtimeblk").alias("arr_time_block"),
    col("cancellationcode").alias("cancellation_code"),
    col("cancelled").cast("bigint"),
    col("carrierdelay").cast("bigint").alias("carrier_delay"),
    col("crsarrtime").alias("crs_arr_time"),
    col("crsdeptime").alias("crs_dep_time"),
    col("crselapsedtime").cast("bigint").alias("crs_elapsed_time"),
    col("dayofmonth").alias("day_of_month"),
    col("dayofweek").alias("day_of_week"),
    col("departuredelaygroups").cast("bigint").alias("dep_delay_groups"),
    col("depdel15").cast("bigint").alias("dep_delay_15"),
    col("depdelay").cast("bigint").alias("dep_delay"),
    col("depdelayminutes").cast("bigint").alias("dep_delay_minutes"),
    col("deptime").cast("bigint").alias("dep_time"),
    col("deptimeblk").alias("dep_time_block"),
    col("dest"),
    col("destairportid").alias("dest_airport_id"),
    col("destairportseqid").alias("dest_airport_seq_id"),
    col("destcitymarketid").alias("dest_city_market_id"),
    col("destcityname").alias("dest_city_name"),
    col("deststate").alias("dest_state"),
    col("deststatefips").alias("dest_state_fips"),
    col("deststatename").alias("dest_state_name"),
    col("destwac").alias("dest_wac"),
    col("distance").cast("bigint"),
    col("distancegroup").alias("distance_group"),
    col("diverted").cast("bigint").alias("diverted"),
    col("dot_id_reporting_airline"),
    col("flight_number_reporting_airline"),
    col("flightdate").cast("date").alias("flight_date"),
    col("flights").cast("bigint"),
    col("iata_code_reporting_airline"),
    col("lateaircraftdelay").cast("bigint").alias("late_aircraft_delay"),
    col("month"),
    col("nasdelay").cast("bigint").alias("nas_delay"),
    col("origin"),
    col("originairportid").alias("origin_airport_id"),
    col("originairportseqid").alias("origin_airport_seq_id"),
    col("origincitymarketid").alias("origin_city_market_id"),
    col("origincityname").alias("origin_city_name"),
    col("originstate").alias("origin_state"),
    col("originstatefips").alias("origin_state_fips"),
    col("originstatename").alias("origin_state_name"),
    col("originwac").alias("origin_wac"),
    col("quarter"),
    col("reporting_airline"),
    col("securitydelay").cast("bigint").alias("security_delay"),
    col("tail_number"),
    col("taxiin").cast("bigint").alias("taxi_in"),
    col("taxiout").cast("bigint").alias("taxi_out"),
    col("weatherdelay").cast("bigint").alias("weather_delay"),
    col("wheelsoff").cast("bigint").alias("wheels_on"),
    col("wheelson").cast("bigint").alias("wheels_off"),
    col("year")
)

In [ ]:
source.printSchema()

## 3. dim_date table

In [ ]:
# get "key"-"key value" from dim_date table
current_dim_date = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_date") \
                    .load().select("id", "flight_date")
# get new data from source dataframe
new_dim_date = source.select("year", "quarter", "month", "day_of_month", "day_of_week", "flight_date").distinct()
# mapping
joined_dim_date = new_dim_date.join(current_dim_date, on="flight_date", how="full")
# fullfill null id
dim_date = joined_dim_date.filter(col("id").isNull()) \
                    .orderBy(col("flight_date").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add date_id to source dataframe
date_id_source = source.join(dim_date.selectExpr("flight_date", "id as date_id"),
                             on="flight_date",
                             how="left")
# get max id of dim_date table
max_date_id = current_dim_date.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_date_id is None:
    max_date_id=-1
# drop old data
to_append_dim_date = dim_date.filter(col("id")>max_date_id)

In [ ]:
# append new data to dim_date table
to_append_dim_date.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_date") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
to_append_dim_date.select("id", "year", "quarter", "month", "day_of_month", "day_of_week", "flight_date") \
            .repartition(1) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_date")

## 4. dim_airline table

In [ ]:
# get "key"-"key value" from dim_airline table
current_dim_airline = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_airline") \
                    .load().select("id", "reporting_airline", "tail_number", "flight_number_reporting_airline")
# get new data from date_id_source dataframe
new_dim_airline = date_id_source.select("reporting_airline", "dot_id_reporting_airline",
                                            "iata_code_reporting_airline", "tail_number",
                                            "flight_number_reporting_airline").distinct()
# mapping
joined_dim_airline = new_dim_airline.join(current_dim_airline,
                                       on=["reporting_airline", "tail_number", "flight_number_reporting_airline"],
                                       how="full")
# fullfill null id
dim_airline = joined_dim_airline.filter(col("id").isNull()) \
                    .orderBy(col("reporting_airline").asc(),
                             col("tail_number").asc(),
                             col("flight_number_reporting_airline").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add airline_id to date_id_source dataframe
airline_id_source = date_id_source.join(dim_airline.selectExpr("reporting_airline", "tail_number",
                                                               "flight_number_reporting_airline", "id as airline_id"),
                                        on=["reporting_airline", "tail_number", "flight_number_reporting_airline"],
                                        how="left")
# get max id of dim_airline table
max_airline_id = current_dim_airline.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_airline_id is None:
    max_airline_id=-1
# drop old data
to_append_dim_airline = dim_airline.filter(col("id")>max_airline_id)

In [ ]:
# append new data to dim_airline table
to_append_dim_airline.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_airline") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
to_append_dim_airline.select("id", "reporting_airline", "dot_id_reporting_airline", "iata_code_reporting_airline",
                             "tail_number", "flight_number_reporting_airline") \
            .repartition(1) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_airline")

## 5. dim_origin table

In [ ]:
# get "key"-"key value" from dim_origin table
current_dim_origin = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_origin") \
                    .load().select("id", "origin_airport_id")
# get new data from airline_id_source dataframe
new_dim_origin = airline_id_source.select("origin_airport_id", "origin_airport_seq_id", "origin_city_market_id",
                               "origin", "origin_city_name", "origin_state", "origin_state_fips",
                              "origin_state_name", "origin_wac").distinct()
# mapping
joined_dim_origin = new_dim_origin.join(current_dim_origin, on="origin_airport_id", how="full")
# fullfill null id
dim_origin = joined_dim_origin.filter(col("id").isNull()) \
                    .orderBy(col("origin_airport_id").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add origin_id to airline_id_source dataframe
origin_id_source = airline_id_source.join(dim_origin.selectExpr("origin_airport_id", "id as origin_id"),
                             on="origin_airport_id",
                             how="left")
# get max id of dim_origin table
max_origin_id = current_dim_origin.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_origin_id is None:
    max_origin_id=-1
# drop old data
to_append_dim_origin = dim_origin.filter(col("id")>max_origin_id)

In [ ]:
# append new data to dim_origin table
to_append_dim_origin.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_origin") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
to_append_dim_origin.select("id", "origin_airport_id", "origin_airport_seq_id", "origin_city_market_id",
                             "origin", "origin_city_name", "origin_state", "origin_state_fips",
                             "origin_state_name", "origin_wac") \
            .repartition(1) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_origin")

## 6. dim_destination table

In [ ]:
# get "key"-"key value" from dim_destination table
current_dim_destination = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_destination") \
                    .load().select("id", "dest_airport_id")
# get new data from origin_id_source dataframe
new_dim_destination = origin_id_source.select("dest_airport_id", "dest_airport_seq_id", "dest_city_market_id",
                               "dest", "dest_city_name", "dest_state", "dest_state_fips",
                              "dest_state_name", "dest_wac").distinct()
# mapping
joined_dim_destination = new_dim_destination.join(current_dim_destination, on="dest_airport_id", how="full")
# fullfill null id
dim_destination = joined_dim_destination.filter(col("id").isNull()) \
                    .orderBy(col("dest_airport_id").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add destination_id to origin_id_source dataframe
destination_id_source = origin_id_source.join(dim_destination.selectExpr("dest_airport_id", "id as destination_id"),
                             on="dest_airport_id",
                             how="left")
# get max id of dim_destination table
max_destination_id = current_dim_destination.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_destination_id is None:
    max_destination_id=-1
# drop old data
to_append_dim_destination = dim_destination.filter(col("id")>max_destination_id)

In [ ]:
# append new data to dim_destination table
to_append_dim_destination.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_destination") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
to_append_dim_destination.select("id", "dest_airport_id", "dest_airport_seq_id", "dest_city_market_id",
                             "dest", "dest_city_name", "dest_state", "dest_state_fips",
                             "dest_state_name", "dest_wac") \
            .repartition(1) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_destination")

## 7. dim_schedule table

In [ ]:
# get "key"-"key value" from dim_schedule table
current_dim_schedule = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_schedule") \
                    .load().select("id", "crs_dep_time", "crs_arr_time", "crs_elapsed_time", "distance")
# get new data from destination_id_source dataframe
new_dim_schedule = destination_id_source.select("crs_dep_time", "dep_time_block", "crs_arr_time",
                               "arr_time_block", "crs_elapsed_time", "distance", "distance_group").distinct()
# mapping
joined_dim_schedule = new_dim_schedule.join(current_dim_schedule,
                                            on=["crs_dep_time", "crs_arr_time", "crs_elapsed_time", "distance"],
                                            how="full")
# fullfill null id
dim_schedule = joined_dim_schedule.filter(col("id").isNull()) \
                    .orderBy(col("crs_dep_time").asc(),
                            col("crs_arr_time").asc(),
                            col("crs_elapsed_time").asc(),
                            col("distance").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add schedule_id to destination_id_source dataframe
schedule_id_source = destination_id_source.join(dim_schedule.selectExpr("crs_dep_time", "crs_arr_time",
                                                                         "crs_elapsed_time", "distance",
                                                                         "id as schedule_id"),
                             on=["crs_dep_time", "crs_arr_time", "crs_elapsed_time", "distance"],
                             how="left")
# get max id of dim_schedule table
max_schedule_id = current_dim_schedule.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_schedule_id is None:
    max_schedule_id=-1
# drop old data
to_append_dim_schedule = dim_schedule.filter(col("id")>max_schedule_id)

In [ ]:
# append new data to dim_schedule table
to_append_dim_schedule.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_schedule") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
to_append_dim_schedule.select("id", "crs_dep_time", "dep_time_block", "crs_arr_time",
                    "arr_time_block", "crs_elapsed_time", "distance", "distance_group") \
            .repartition(1) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_schedule")

## 8. dim_cancellations_and_diversions table

In [ ]:
# get "key"-"key value" from dim_cancellations_and_diversions table
current_dim_cad = spark.read \
                    .format("bigquery") \
                    .options(parentProject="test-373705") \
                    .options(table="flight_delay.dim_cancellations_and_diversions") \
                    .load().select("id", "cancelled", "cancellation_code", "diverted")
# get new data from schedule_id_source dataframe
new_dim_cad = schedule_id_source.select("cancelled", "cancellation_code", "diverted").distinct()
# mapping
joined_dim_cad = new_dim_cad.join(current_dim_cad,
                                  on=["cancelled", "cancellation_code", "diverted"],
                                  how="full")
# fullfill null id
dim_cad = joined_dim_cad.filter(col("id").isNull()) \
                    .orderBy(col("cancelled").asc(),
                            col("cancellation_code").asc(),
                            col("diverted").asc()) \
                    .withColumn("id", monotonically_increasing_id())
# add cancellations_and_diversions_id to schedule_id_source dataframe
completed_source = schedule_id_source.join(dim_cad.selectExpr("cancelled", "cancellation_code","diverted",
                                                                     "id as cancellations_and_diversions_id"),
                                           on=["cancelled", "cancellation_code", "diverted"],
                                           how="left")
# get max id of dim_cancellations_and_diversions table
max_cad_id = current_dim_cad.agg(max("id")).collect()[0][0]
# ensure max id have a value
if max_cad_id is None:
    max_cad_id=-1
# drop old data
to_append_dim_cad = dim_cad.filter(col("id")>max_cad_id)

In [ ]:
# append new data to dim_cancellations_and_diversions table
to_append_dim_cad.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.dim_cancellations_and_diversions") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
to_append_dim_cad.select("id", "cancelled", "cancellation_code","diverted") \
            .repartition(1) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/dim_cancellations_and_diversions")

## 9. fact_records table

In [ ]:
# select necessary columns for fact_records table
fact_records = completed_source.select("id", "date_id", "airline_id", "origin_id", "destination_id", "schedule_id",
                                      "cancellations_and_diversions_id", "dep_time", "dep_delay", "dep_delay_minutes",
                                      "dep_delay_15", "dep_delay_groups", "taxi_out", "taxi_in", "wheels_off",
                                      "wheels_on", "arr_time", "arr_delay", "arr_delay_minutes", "arr_delay_15",
                                      "arr_delay_groups", "actual_elapsed_time", "air_time", "flights",
                                      "carrier_delay", "weather_delay", "nas_delay", "security_delay",
                                       "late_aircraft_delay")

In [ ]:
# append new data to fact_records table
fact_records.orderBy(col("id").asc()) \
            .write \
            .format("bigquery") \
            .options(parentProject="test-373705") \
            .options(table="flight_delay.fact_records") \
            .option("writeMethod", "direct") \
            .mode("append") \
            .save()

In [ ]:
# export new data to csv
fact_records.repartition(2) \
            .orderBy(col("id").asc()) \
            .write \
            .format("csv") \
            .options(header="True") \
            .save("../../flight-delay/bigquery/fact_records")

In [ ]:
spark.stop()